# Web Scraping with Beautiful Soup

In [1]:
# import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser

In [2]:
# initialize browser
executable_path = {'executable_path': '../chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [3]:
# scrape latest news article title and headline

# store full url
news_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# visit mars news website
browser.visit(news_url)

# parse through html
news_soup = bs(browser.html, 'html.parser')

# store title and headline
news_title = news_soup.find_all('div', class_='content_title')[0].text
news_p = news_soup.find_all('div', class_='article_teaser_body')[0].text

# print title and headline
print(news_title)
print(news_p)

Space Samples Link NASA's Apollo 11 and Mars 2020
While separated by half a century, NASA's Apollo 11 and Mars 2020 missions share the same historic goal: returning samples to Earth.


### JPL Mars Space Images - Featured Image

In [4]:
# scrape featured_image_url

# store full and base urls
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
image_base_url = 'https://www.jpl.nasa.gov'

# visit site
browser.visit(image_url)

# parse through html
image_soup = bs(browser.html, 'html.parser')

# previous attempt to click FULL IMAGE was having hiccups so tried to parse through style content
partial_image_url  = image_soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]

# combine base url with partial url
featured_image_url = image_base_url + partial_image_url

# print url
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA20263-1920x1200.jpg'

### Mars Weather

In [5]:
# scrape latest weather tweet

# store full and base urls
weather_url = 'https://twitter.com/marswxreport?lang=en'

# visit site
browser.visit(weather_url)

# parse through html
weather_soup = bs(browser.html, 'html.parser')

# store latest tweet
latest_weather_tweet = weather_soup.find('p', class_='TweetTextSize').text

# print latest tweet
latest_weather_tweet

'InSight sol 251 (2019-08-11) low -101.0ºC (-149.7ºF) high -26.5ºC (-15.8ºF)\nwinds from the SSE at 4.1 m/s (9.2 mph) gusting to 17.5 m/s (39.1 mph)\npressure at 7.60 hPapic.twitter.com/9mgFzHl8t3'

### Mars Facts

In [6]:
# scrape table with facts about mars

# store full url
facts_url = 'https://space-facts.com/mars/'

# reading html into a dataframe
facts_table = pd.read_html(facts_url)

# store only mars facts
mars_table = facts_table[1]

# renaming columns
facts_mapping = {0:'Description', 1:'Value'}
mars_table = mars_table.rename(columns=facts_mapping)

# saving as html table format
mars_table.to_html('mars_table.html', index=False)

# print data frame
print(mars_table)

            Description                          Value
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers


### Mars Hemispheres

In [7]:
# scrape hemisphere images

# store full and base urls
astro_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
astro_base_url = "https://astrogeology.usgs.gov"

# visit url
browser.visit(astro_url)
astro_soup = bs(browser.html, 'html.parser')

# creating empty lists to append scraped data
hemi_image_list = []
title_list = []
url_list = []

# loop through links and avoid two links that are the same
div_list = astro_soup.find_all('a', class_='itemLink')
for link in div_list:
    image_url = link.get('href')
    if image_url not in hemi_image_list:
        hemi_image_list.append(image_url)
        browser.visit(astro_base_url + image_url)
        title_list.append(browser.find_by_tag('h2').text)
        browser.find_link_by_text('Sample').click()
        
for i in range(4,0,-1):
    url_list.append(browser.windows[i].url)
    
# close browser
browser.quit()

In [8]:
# create list of hemisphere titles and hemisphere urls in dictionaries
featured_hemisphere_list = [{'title': title_list[0], 'img_url': url_list[0]},
                            {'title': title_list[1], 'img_url': url_list[1]},
                            {'title': title_list[2], 'img_url': url_list[2]},
                            {'title': title_list[3], 'img_url': url_list[3]}]
featured_hemisphere_list

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]